# Quantize OPT350m model with Post-Training Optimization Tool ​in OpenVINO™
This tutorial demonstrates how to apply `INT8` quantization to the [OPT350m](https://huggingface.co/facebook/opt-350m), using the [Post-Training Optimization Tool API](https://docs.openvino.ai/latest/pot_compression_api_README.html) (part of the [OpenVINO Toolkit](https://docs.openvino.ai/)). [Microsoft Research Paraphrase Corpus (MRPC)](https://www.microsoft.com/en-us/download/details.aspx?id=52398) dataset is used for quantization.
Structure of the notebook is as follows:
- Download and prepare the OPT350m model and MRPC dataset.
- Define data loading and accuracy validation functionality. Accuracy checking was not performed due to memory limits
- Prepare the model for quantization.
- Run optimization pipeline.
- Load and test quantized model.
- Compare the performance of the original, converted and quantized models.

## Imports

In [ ]:
import os
import sys
import time
import warnings
import typing as t
from pathlib import Path
from zipfile import ZipFile

import numpy as np
import torch
from torch import nn
from addict import Dict
from openvino import runtime as ov
from openvino.tools.pot import DataLoader as POTDataLoader
from openvino.tools.pot import Metric, IEEngine, load_model, save_model, compress_model_weights, create_pipeline
from torch.utils.data import TensorDataset
from transformers import AutoTokenizer, OPTModel
from transformers import (
    glue_convert_examples_to_features as convert_examples_to_features,
)
from transformers import glue_output_modes as output_modes
from transformers import glue_processors as processors

sys.path.append("../utils")

## Settings

In [5]:
# Set the data and model directories, source URL and the filename of the model.
DATA_DIR = "."
MODEL_DIR = "."

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

## Prepare the Model
Perform the following:
- Download and unpack pre-trained BERT model for MRPC by PyTorch.
- Convert the model to the ONNX.
- Run Model Optimizer to convert the model from the ONNX representation to the OpenVINO Intermediate Representation (OpenVINO IR)

Import all dependencies to load the original PyTorch model and convert it to the ONNX representation.

In [6]:
BATCH_SIZE = 1
MAX_SEQ_LENGTH = 128


def export_model_to_onnx(model: nn.Module, path: t.Union[str, Path]) -> None:
    
    '''
    Converts pytorch model to onnx file

    Parameters
    ----------
    model : pytorch model to be converted to onnx
    path : path to onnx file
    '''

    with torch.no_grad():
        default_input = torch.ones(1, MAX_SEQ_LENGTH, dtype=torch.int64)
        
        inputs = {
            "input_ids": default_input,
            "attention_mask": default_input,
        }
        
        symbolic_names = {0: "batch_size", 1: "max_seq_len"}

        torch.onnx.export(
            model,
            (inputs["input_ids"], inputs["attention_mask"]),
            path,
            opset_version=11,
            do_constant_folding=True,
            input_names=["input_ids", "attention_mask"],
            output_names=["output"],
            dynamic_axes={
                "input_ids": symbolic_names,
                "attention_mask": symbolic_names,
            },
        )
        print("ONNX model saved to {}".format(path))


torch_model = OPTModel.from_pretrained("facebook/opt-350m")
onnx_model_path = Path(MODEL_DIR) / "opt350m.onnx"

if not onnx_model_path.exists():
    export_model_to_onnx(torch_model, onnx_model_path)

In [7]:
## If RAM is not enough for quantization, consider deleting unnecessary objects

# del torch_model
# import gc
# gc.collect()

In [12]:
# This is to check if created onnx model is working, at least if inference is successful

import onnxruntime as ort

so = ort.SessionOptions()
so.log_severity_level = 3

rt_sess = ort.InferenceSession(str(onnx_model_path), sess_options=so, providers=['CPUExecutionProvider'])
input_name = rt_sess.get_inputs()[0].name
input_shape =rt_sess.get_inputs()[0].shape
print(input_name, input_shape)


input_name = rt_sess.get_inputs()[1].name
input_shape =rt_sess.get_inputs()[1].shape
print(input_name, input_shape)

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")
inputs = tokenizer("Hello, my dog is cute", return_tensors="np")
x = dict(**inputs)
onnx_out = rt_sess.run(None, x)

onnx_out[1].shape, len(onnx_out)

input_ids ['batch_size', 'max_seq_len']
attention_mask ['batch_size', 'max_seq_len']


((1, 16, 7, 64), 49)

## Convert the ONNX Model to OpenVINO IR

In [13]:
ir_model_xml = onnx_model_path.with_suffix(".xml")
ir_model_bin = onnx_model_path.with_suffix(".bin")

# Convert the ONNX model to OpenVINO IR FP32.
if not ir_model_xml.exists():
    !mo --input_model $onnx_model_path --output_dir $MODEL_DIR --model_name $ir_model_xml.stem --input input_ids,attention_mask --input_shape "[1,128],[1,128]" --output output

## Prepare MRPC Task Dataset

To run this tutorial, you will need to download the General Language Understanding Evaluation  (GLUE) data for the MRPC task from HuggingFace. Use the code below to download a script that fetches the MRPC dataset.

In [9]:
!wget https://raw.githubusercontent.com/huggingface/transformers/f98ef14d161d7bcdc9808b5ec399981481411cc1/utils/download_glue_data.py

--2023-03-22 22:29:50--  https://raw.githubusercontent.com/huggingface/transformers/f98ef14d161d7bcdc9808b5ec399981481411cc1/utils/download_glue_data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8209 (8,0K) [text/plain]
Saving to: ‘download_glue_data.py’

download_glue_data. 100%[===================>]   8,02K  --.-KB/s    in 0,02s   

2023-03-22 22:29:51 (344 KB/s) - ‘download_glue_data.py’ saved [8209/8209]



In [10]:
from download_glue_data import format_mrpc

format_mrpc(DATA_DIR, "")

Processing MRPC...
Local MRPC data not specified, downloading data from https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt
	Completed!


## Define DataLoader for POT
In this step, you define `DataLoader` based on POT API. It will be used to collect statistics for quantization and run model evaluation. 
Use helper functions from the HuggingFace Transformers to do the data preprocessing. It takes raw text data and encodes sentences and words, producing three model inputs. 
For more details about the data preprocessing and tokenization, refer to this [description](https://medium.com/@dhartidhami/understanding-bert-word-embeddings-7dc4d2ea54ca).

In [16]:
class MRPCDataLoader(POTDataLoader):
    # Required methods
    def __init__(self, config: Dict):
        """Constructor
        :param config: data loader specific config
        """
        if not isinstance(config, Dict):
            config = Dict(config)
        super().__init__(config)
        self._task = config["task"].lower()
        self._model_dir = config["model_dir"]
        self._data_dir = config["data_source"]
        self._batch_size = config["batch_size"]
        self._max_length = config["max_length"]
        self.examples = []
        self._prepare_dataset()

    def __len__(self)-> int:
        """Returns size of the dataset"""
        return len(self.dataset)

    def __getitem__(self, index: int)-> t.Tuple[t.Tuple, t.Dict]:
        """
        Returns annotation, data and metadata at the specified index.
        Possible formats:
        (index, annotation), data
        (index, annotation), data, metadata
        """
        if index >= len(self):
            raise IndexError

        batch = self.dataset[index]
        batch = tuple(t.detach().cpu().numpy() for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1]}
        labels = batch[2]
        return (index, labels), inputs

    # Methods specific to the current implementation
    def _prepare_dataset(self):
        """Prepare dataset"""
        tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m", do_lower_case=True)
        processor = processors[self._task]()
        output_mode = output_modes[self._task]
        label_list = processor.get_labels()
        examples = processor.get_dev_examples(self._data_dir)
        features = convert_examples_to_features(
            examples,
            tokenizer,
            label_list=label_list,
            max_length=self._max_length,
            output_mode=output_mode,
        )
        all_input_ids = torch.unsqueeze(torch.tensor([f.input_ids for f in features], dtype=torch.long), 1)
        all_attention_mask = torch.unsqueeze(torch.tensor([f.attention_mask for f in features], dtype=torch.long), 1)
        all_labels = torch.unsqueeze(torch.tensor([f.label for f in features], dtype=torch.long), 1)
        self.dataset = TensorDataset(
            all_input_ids, all_attention_mask, all_labels
        )
        self.examples = examples

## Define Accuracy Metric Calculation
In this step the `Metric` interface for MRPC task metrics is implemented. It is used for validating the accuracy of the models.

In [17]:
class Accuracy(Metric):

    # Required methods
    def __init__(self):
        super().__init__()
        self._name = "Accuracy"
        self._matches = []

    @property
    def value(self) -> t.Dict:
        """Returns accuracy metric value for the last model output."""
        return {self._name: self._matches[-1]}

    @property
    def avg_value(self) -> t.Dict:
        """Returns accuracy metric value for all model outputs."""
        return {self._name: np.ravel(self._matches).mean()}

    def update(self, output: np.ndarray, target: np.ndarray):
        """
        Updates prediction matches.

        :param output: model output
        :param target: annotations
        """
        if len(output) > 1:
            raise Exception(
                "The accuracy metric cannot be calculated " "for a model with multiple outputs"
            )
        output = np.argmax(output)
        match = output == target[0]
        self._matches.append(match)

    def reset(self):
        """
        Resets collected matches
        """
        self._matches = []

    def get_attributes(self) -> t.Dict :
        """
        Returns a dictionary of metric attributes {metric_name: {attribute_name: value}}.
        Required attributes: 'direction': 'higher-better' or 'higher-worse'
                             'type': metric type
        """
        return {self._name: {"direction": "higher-better", "type": "accuracy"}}

## Run Quantization Pipeline
Define a configuration for the quantization pipeline and run it. Keep in mind that built-in `IEEngine` implementation of `Engine` interface from the POT API for model inference is used here.

In [23]:
warnings.filterwarnings("ignore")  # Suppress accuracychecker warnings.

model_config = Dict({"model_name": "opt350m_defquant", "model": ir_model_xml, "weights": ir_model_bin})
engine_config = Dict({"device": "CPU"})
dataset_config = {
    "task": "mrpc",
    "data_source": os.path.join(DATA_DIR, "MRPC"),
    "model_dir": os.path.join(MODEL_DIR, "MRPC"),
    "batch_size": BATCH_SIZE,
    "max_length": MAX_SEQ_LENGTH,
}

# We are using DefaultQuantization, and target device is any. Target device can be replaced to CPU
# stat_subset_size is set to 2 to lower the memory consumption

algorithms = [
    {
        "name": "DefaultQuantization",
        "params": {
            "target_device": "ANY",
            "model_type": "transformer",
            "preset": "performance",
            "stat_subset_size": 2,
        },
    }
]


# Step 1: Load the model.
model = load_model(model_config=model_config)

# Step 2: Initialize the data loader.
data_loader = MRPCDataLoader(config=dataset_config)

# Step 3 (Optional. Required for AccuracyAwareQuantization): Initialize the metric.
# metric = Accuracy()

# Step 4: Initialize the engine for metric calculation and statistics collection.
# engine = IEEngine(config=engine_config, data_loader=data_loader, metric=metric)
engine = IEEngine(config=engine_config, data_loader=data_loader)

# Step 5: Create a pipeline of compression algorithms.
pipeline = create_pipeline(algo_config=algorithms, engine=engine)

# Step 6 (Optional): Evaluate the original model. Print the results.
# fp_results = pipeline.evaluate(model=model)
# if fp_results:
#     print("FP32 model results:")
#     for name, value in fp_results.items():
#         print(f"{name}: {value:.5f}")

In [20]:
# To remove unnecessary objects
import gc
gc.collect()

123443

In [25]:
# Step 7: Execute the pipeline.
warnings.filterwarnings("ignore")  # Suppress accuracychecker warnings.
print(
    f"Quantizing model with {algorithms[0]['params']['preset']} preset and {algorithms[0]['name']}"
)
start_time = time.perf_counter()
compressed_model = pipeline.run(model=model)

del model
gc.collect()

end_time = time.perf_counter()
print(f"Quantization finished in {end_time - start_time:.2f} seconds")

# Step 8 (Optional): Compress model weights to quantized precision
#                    in order to reduce the size of the final .bin file.
compress_model_weights(model=compressed_model)

# Step 9: Save the compressed model to the desired path.
compressed_model_paths = save_model(model=compressed_model, save_path=MODEL_DIR, model_name="opt350_defquant")

compressed_model_xml = compressed_model_paths[0]["model"]

Quantizing model with performance preset and DefaultQuantization
Quantization finished in 372.00 seconds


## Load and Test OpenVINO Model

To load and test converted model, perform the following:
* Load the model and compile it for CPU.
* Prepare the input.
* Run the inference.
* Get the answer from the model output.

In [26]:
core = ov.Core()

# Read the model from files.
model = core.read_model(model=compressed_model_xml)

# Assign dynamic shapes to every input layer.
for input_layer in model.inputs:
    input_shape = input_layer.partial_shape
    input_shape[1] = -1
    model.reshape({input_layer: input_shape})

# Compile the model for a specific device.
compiled_model_int8 = core.compile_model(model=model, device_name="CPU")

output_layer = compiled_model_int8.outputs[0]

The Data Loader returns a pair of sentences (indicated by `sample_idx`) and the inference compares these sentences and outputs whether their meaning is the same. You can test other sentences by changing `sample_idx` to another value (from 0 to 407).

## Compare Performance of the Original, Converted and Quantized Models

Compare the original PyTorch model with OpenVINO converted and quantized models (`FP32`, `INT8`) to see the difference in performance. It is expressed in Sentences Per Second (SPS) measure, which is the same as Frames Per Second (FPS) for images.

In [27]:
model = core.read_model(model=ir_model_xml)

# Assign dynamic shapes to every input layer.
for input_layer in model.inputs:
    input_shape = input_layer.partial_shape
    input_shape[1] = -1
    model.reshape({input_layer: input_shape})

# Compile the model for a specific device.
compiled_model_fp32 = core.compile_model(model=model, device_name="CPU")

In [20]:
num_samples = 50
inputs = data_loader[0][1]

with torch.no_grad():
    start = time.perf_counter()
    for _ in range(num_samples):
        torch_model(torch.as_tensor(list(inputs.values())).squeeze())
    end = time.perf_counter()
    time_torch = end - start
print(
    f"PyTorch model on CPU: {time_torch / num_samples:.3f} seconds per sentence, "
    f"SPS: {num_samples / time_torch:.2f}"
)

start = time.perf_counter()
for _ in range(num_samples):
    compiled_model_fp32(inputs)
end = time.perf_counter()
time_ir = end - start
print(
    f"IR FP32 model in OpenVINO Runtime/CPU: {time_ir / num_samples:.3f} "
    f"seconds per sentence, SPS: {num_samples / time_ir:.2f}"
)

start = time.perf_counter()
for _ in range(num_samples):
    compiled_model_int8(inputs)
end = time.perf_counter()
time_ir = end - start
print(
    f"OpenVINO IR INT8 model in OpenVINO Runtime/CPU: {time_ir / num_samples:.3f} "
    f"seconds per sentence, SPS: {num_samples / time_ir:.2f}"
)

PyTorch model on CPU: 1.240 seconds per sentence, SPS: 0.81
IR FP32 model in OpenVINO Runtime/CPU: 0.443 seconds per sentence, SPS: 2.26
OpenVINO IR INT8 model in OpenVINO Runtime/CPU: 0.165 seconds per sentence, SPS: 6.06


Finally, measure the inference performance of OpenVINO `FP32` and `INT8` models. For this purpose, use [Benchmark Tool](https://docs.openvino.ai/latest/openvino_inference_engine_tools_benchmark_tool_README.html) in OpenVINO.

> **Note**: The `benchmark_app` tool is able to measure the performance of the OpenVINO Intermediate Representation (OpenVINO IR) models only. For more accurate performance, run `benchmark_app` in a terminal/command prompt after closing other applications. Run `benchmark_app -m model.xml -d CPU` to benchmark async inference on CPU for one minute. Change `CPU` to `GPU` to benchmark on GPU. Run `benchmark_app --help` to see an overview of all command-line options.

In [21]:
# Inference FP32 model (OpenVINO IR)
! benchmark_app -m $ir_model_xml -d CPU -api sync

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2022.3.0-9052-9752fafe8eb-releases/2022/3
[ INFO ] 
[ INFO ] Device info:
[ INFO ] CPU
[ INFO ] Build ................................. 2022.3.0-9052-9752fafe8eb-releases/2022/3
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[ WARNING ] Performance hint was not explicitly specified in command line. Device(CPU) performance hint will be set to LATENCY.
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 689.95 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     input_ids (node: input_ids) : i64 / [...] / [1,128]
[ INFO ]     attention_mask , /decoder/embed_positions/Cast_output_0 (node: attention_mask) : i64 / [...] / [1,128]
[ INFO ] Model outputs:
[ INFO ]     output (node: output) : f32 / [...] / [1,128,512]
[Step 5/11] Resi

In [23]:
# Inference INT8 model (OpenVINO IR)
! benchmark_app -m opt350_defquant.xml -d CPU -api sync

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2022.3.0-9052-9752fafe8eb-releases/2022/3
[ INFO ] 
[ INFO ] Device info:
[ INFO ] CPU
[ INFO ] Build ................................. 2022.3.0-9052-9752fafe8eb-releases/2022/3
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[ WARNING ] Performance hint was not explicitly specified in command line. Device(CPU) performance hint will be set to LATENCY.
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 418.53 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     input_ids (node: input_ids) : i64 / [...] / [1,128]
[ INFO ]     attention_mask , /decoder/embed_positions/Cast_output_0 (node: attention_mask) : i64 / [...] / [1,128]
[ INFO ] Model outputs:
[ INFO ]     output (node: output) : f32 / [...] / [1,128,512]
[Step 5/11] Resi